#RAG(Retriveal-Augmented-Generation): 검색 증강 생성
    자연어 처리(NLP) 및 기계 학습 분야, 특히 챗복이나 질문-응답 시스템과 같은 고급 언어 모델을 구축하는 데 사용되는 기술

#NLP의 두 가지 주요 구성 요소인 정보 검색과 응답 생성을 결합
    검색 부분은 관련 정보를 찾기 위해 대규모 데이터베이스나 문서 컬렉션을 검색하는 과정을 포함
    생성 부분은 검색된 정보를 바탕으로 일관되고 맥락적으로 적절한 텍스트를 생성하는 과정

#작동 방식
    1. 질문이나 프롬프트가 주어지면 모델은 먼저 질문에 대한 답변을 제공하는 데 유용한 정보를 포함할 수 있는 관련 문서나 텍스트를 검색
    2. 검색된 정보를 생성 모델에 공급하여 일관된 응답을 합성
    => 질문 / 질문을 할 때 저장된 vector에서 단어를 탐색(유, 무 아님) -> 관련 문서를 prompt로 보낸다 -> model로 해당 prompt를 전달

#장점:
    모델이 외부 지식을 활용할 수 있게 하여 보다 정확하고 상세하며 맥락적으로 관련된 답변을 제공
    => 특정 지식이나 사실적 정보가 필요한 질문에 특히 유용

#응용 분야:
    챗봇, 질문-응답 시스템 및 정확하고 맥락적으로 관련된 정보를 제공하는 것이 중요한 다른 AI 도구와 같은 다양한 응용 분야에 사용
    다양한 주제와 데이터를 기반으로 이해하고 응답을 생성해야 하는 상황에서 유용

data Retrieve
    Source -> Load -> Transform -> Embed -> Storage -> Retrieve
    소스에서 데이터 load 후, 데이터 분할하며 변환 -> 변환한 데이터를 임베드(텍스트에서 컴퓨터가 이해할 수 있는 숫자로 변환) -> 저장 / 검색

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
)
# chunk_size=200, chunk_overlap=50
# chunk_overlap: 문장이나 문단을 분할할 때 앞 조각 끝 일부분을 가져오게 만든다. -> 중복이 생긴다.
# length_function=len: 얼마나 많은 글자가 있는지 세 준다.

# loader = TextLoader("./files/chapter_one.txt")
# loader = PyPDFLoader("./files/chapter_one.pdf")
# docs = loader.load()

loader = UnstructuredFileLoader("./files/chapter_one.docx")

# len(loader.load_and_split(text_splitter=splitter))
loader.load_and_split(text_splitter=splitter)



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
    
)
# length_function=len: 얼마나 많은 글자가 있는지 세 준다.
# token은 문자와 같은 의미가 아니다 -> 문자 두, 세 개를 한 개의 token으로 취급하기도 한다.

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)

Vectors
    vecroization(벡터화) / 우리가 만든 문서마다 각각의 벡터를 만든다.
    word(단어)에 embed 작업을 한다.

Embed
    Embedding = 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업
